<a href="https://colab.research.google.com/github/kkruglik/nlp_notebooks/blob/main/rut5_multitask_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Демо

In [ ]:
!pip install transformers sentencepiece --quiet

     |████████████████████████████████| 2.9 MB 8.4 MB/s 
     |████████████████████████████████| 1.2 MB 34.5 MB/s 
     |████████████████████████████████| 636 kB 39.2 MB/s 
     |████████████████████████████████| 3.3 MB 42.0 MB/s 
     |████████████████████████████████| 895 kB 20.0 MB/s 
     |████████████████████████████████| 56 kB 4.5 MB/s 
     |████████████████████████████████| 109 kB 50.7 MB/s 
     |████████████████████████████████| 546 kB 44.4 MB/s 


In [ ]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
model_name = "cointegrated/rut5-base-multitask"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

def generate(text, **kwargs):
    inputs = tokenizer(text, return_tensors='pt')
    with torch.no_grad():
        hypotheses = model.generate(**inputs, num_beams=5, **kwargs)
    return tokenizer.decode(hypotheses[0], skip_special_tokens=True)

In [ ]:
# print(generate('translate ru-en | Каждый охотник желает знать, где сидит
print(generate(
    'translate ru-en | Каждый охотник желает знать, где сидит фазан.'))
# Each hunter wants to know, where he is.

Each hunter wants to know, where he is.


In [ ]:
print(generate(
    'translate en-ru | Every hunter wants to know where the duck is.'))
# Все охотники хотят знать, где находится птица.

Все охотники хотят знать, где находится птица.


In [ ]:
print(generate(
    'paraphrase | Каждый охотник желает знать, где сидит фазан.', 
    encoder_no_repeat_ngram_size=1, repetition_penalty=0.5, 
    no_repeat_ngram_size=1
))
# В любом случае каждый рыбак мечтает познакомиться со своей фермой

В любом случае каждый рыбак мечтает познакомиться со своей фермой


In [ ]:
print(generate('fill | Каждый охотник _3_, где сидит фазан.'))
# смотрит на озеро

print(generate('assemble | охотник каждый знать фазан сидит'))
# Каждый охотник знает, что фазан сидит.

смотрит на озеро
Каждый охотник знает, что фазан сидит.


In [ ]:
# print(generate('translate ru-en | Каждый охотник желает знать, где сидит

print(generate(
    'simplify | Местным продуктом-специалитетом с защищённым ' \
    'географическим наименованием по происхождению считается ' \
    'люнебургский степной барашек.', max_length=32))
# Местным продуктом-специалитетом считается люнебургский степной барашек.

Местным продуктом-специалитетом считается люнебургский степной барашек.


In [ ]:
print(generate('reply | Помогите мне закадрить девушку'))
# Что я хочу?

print(generate('answer | Помогите мне закадрить девушку'))
# я хочу познакомиться с девушкой!!!!!!!!

Что я хочу?
я хочу познакомиться с девушкой!!!!!!!!


In [ ]:
print(generate(
    'comprehend | На фоне земельного конфликта между владельцами овец и ' \
    'ранчеро разворачивается история любви овцевода Моргана Лейна, ' \
    'прибывшего в США из Австралии, и Марии Синглетон, владелицы ' \
    'богатого скотоводческого ранчо. Вопрос: откуда приехал Морган?'))
# из Австралии

из Австралии


In [ ]:
print(generate(
    'ask | На фоне земельного конфликта между владельцами овец и ' \
    'ранчеро разворачивается история любви овцевода Моргана Лейна, ' \
    'прибывшего в США из Австралии, и Марии Синглетон, владелицы ' \
    'богатого скотоводческого ранчо.', max_length=32))
# Что разворачивается на фоне земельного конфликта 
# между владельцами овец и ранчеро?

Что разворачивается на фоне земельного конфликта между владельцами овец и ранчеро?


In [ ]:
print(generate(
    'headline | На фоне земельного конфликта между владельцами овец ' \
    'и ранчеро разворачивается история любви овцевода Моргана Лейна, ' \
    'прибывшего в США из Австралии, и Марии Синглетон, владелицы ' \
    'богатого скотоводческого ранчо.', max_length=32))
# На фоне земельного конфликта разворачивается история любви 
# овцевода Моргана Лейна и Марии Синглетон

На фоне земельного конфликта разворачивается история любви овцевода Моргана Лейна и Марии Синглетон


# Пример обучения на свою задачу

Мы обучим нашу модель T5 на задачу генерации ответа на вопрос кроссвордно-викторинного типа. 

In [ ]:
!pip install transformers sentencepiece 

In [ ]:
!gdown --id 13n_UBlkCyvy28GDa-tTfR2KCtaIHyjBB

Downloading...
From: https://drive.google.com/uc?id=13n_UBlkCyvy28GDa-tTfR2KCtaIHyjBB
To: /content/quiz.tsv
20.9MB [00:00, 79.2MB/s]


In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
import random
from tqdm.auto import tqdm, trange
import os
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('quiz.tsv', sep='\t')

In [ ]:
pd.options.display.max_colwidth = 500

Примеры вопросов и ответов выглядят как-то так. 

In [ ]:
df.sample(5)

,q,a,d,idx,page,categ
89557,Массовое собрание прибывших из разных мест членов спортивной организации,слет,[],81978,80280,24
52362,Cyмкa для мaгaзинoв c пaтpoнaми,подсумок,[],44623,43090,24
103285,Подземная горная выработка с забоем большой протяженности,лава,[],95750,94010,24
128640,"Это слово до XI века имело значение суд, правосудие, приговор. А сейчас это совсем не суд, а скорее суд божий. Что это?",судьба,[],121250,119360,24
52785,Cкaзкa K. И. Чyкoвcкoгo,Мойдодыр,[],45046,43510,24


Разобьем выборку на обучающую и тестовую, и обучающую превратим в список пар. 

In [ ]:
df_train, df_test = train_test_split(df.dropna(), test_size=0.5, random_state=1)
pairs = df_train[['q', 'a']].values.tolist()

Инициализируем модель из предобученной

In [ ]:
raw_model = 'cointegrated/rut5-base-multitask' 
model = T5ForConditionalGeneration.from_pretrained(raw_model).cuda();
tokenizer = T5Tokenizer.from_pretrained(raw_model)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

Обучаем модель! Мы пройдём три эпохи, т.е. покажем данные модели по три раза.

На GPU с размером батча 16 одна эпоха должна занять где-то 15 минут. 

In [ ]:
batch_size = 16  # сколько примеров показывем модели за один шаг
report_steps = 200  # раз в сколько шагов печатаем результат
epochs = 3  # сколько раз мы покажем данные модели

In [ ]:
model.train()
losses = []
for epoch in range(epochs):
    print('EPOCH', epoch)
    random.shuffle(pairs)
    for i in trange(0, int(len(pairs) / batch_size)):
        batch = pairs[i * batch_size: (i + 1) * batch_size]
        x = tokenizer([p[0] for p in batch], return_tensors='pt', padding=True).to(model.device)
        y = tokenizer([p[1] for p in batch], return_tensors='pt', padding=True).to(model.device)
        y.input_ids[y.input_ids == 0] = -100
        loss = model(
            input_ids=x.input_ids,
            attention_mask=x.attention_mask,
            labels=y.input_ids,
            decoder_attention_mask=y.attention_mask,
            return_dict=True
        ).loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        losses.append(loss.item())
        if i % report_steps == 0:
            print('step', i, 'loss', np.mean(losses[-report_steps:]))

EPOCH 1


  0%|          | 0/4892 [00:00<?, ?it/s]

step 0 loss 3.8470616340637207
step 200 loss 4.236956038475037
step 400 loss 4.243761397600174
step 600 loss 4.185914375782013
step 800 loss 4.21162052154541
step 1000 loss 4.204087338447571
step 1200 loss 4.202062028646469
step 1400 loss 4.189540849924088
step 1600 loss 4.168862791061401
step 1800 loss 4.174042043685913
step 2000 loss 4.123842047452927
step 2200 loss 4.086935578584671
step 2400 loss 4.143192913532257
step 2600 loss 4.176858893632889
step 2800 loss 4.13883326292038
step 3000 loss 4.160567367076874
step 3200 loss 4.092823115587234
step 3400 loss 4.107407578229904
step 3600 loss 4.153625081777573
step 3800 loss 4.086228107213974
step 4000 loss 4.132798237800598
step 4200 loss 4.074807376861572
step 4400 loss 4.124523136615753
step 4600 loss 4.053861576318741
step 4800 loss 4.07164126753807
EPOCH 2


  0%|          | 0/4892 [00:00<?, ?it/s]

step 0 loss 4.070417506694794
step 200 loss 4.070597746372223
step 400 loss 4.094182950258255
step 600 loss 4.063615082502365
step 800 loss 3.967441118955612
step 1000 loss 4.011284437179565
step 1200 loss 4.020660309791565
step 1400 loss 3.9995454287528993
step 1600 loss 3.998213469982147
step 1800 loss 3.971472758054733
step 2000 loss 4.004275197982788
step 2200 loss 3.9787186360359192
step 2400 loss 3.954311252832413
step 2600 loss 3.972656538486481
step 2800 loss 3.9850507843494416
step 3000 loss 3.9864893901348113
step 3200 loss 3.918144265413284
step 3400 loss 3.98400639295578
step 3600 loss 3.972563716173172
step 3800 loss 3.951940791606903
step 4000 loss 3.968209079504013
step 4200 loss 3.967874040603638
step 4400 loss 3.9230076920986177
step 4600 loss 3.9887063550949096
step 4800 loss 3.957128328084946


Это - функция для генерации ответа

In [ ]:
model.eval()

def answer(x, **kwargs):
    inputs = tokenizer(x, return_tensors='pt').to(model.device) 
    with torch.no_grad():
        hypotheses = model.generate(**inputs, **kwargs)
    return tokenizer.decode(hypotheses[0], skip_special_tokens=True)

Посмотрим, насколько хорошо модель выучила свою тренировочную выборку. 

In [ ]:
sample = df_train.sample(5)
for i, row in sample.iterrows():
    print(row.q)
    print('answer:', row.a)
    print('model: ', answer(row.q))
    print('---')

Какое животное раньше называли камелопардом?
answer: Жираф
model:  акула
---
Грамматическая категория глагола, выражающая отношение действия к действительности (в лингвистике)
answer: наклонение
model:  действие
---
О чём поется в песне Greenday – «Wake Me Up When September Ends» (Разбуди меня, когда сентябрь кончится)?
answer: О смерти его отца
model:  о лете
---
Соседка Земли по Солнечной системе
answer: Венера
model:  Африка
---
Отношение размеров на чертеже, карте и т. п. к действительным размерам на местности, предмете
answer: масштаб
model:  пропорциональность
---


Теперь посмотрим, насколько хорошо модель справляется с ответами на вопросы, которые она не видела. 

In [ ]:
sample = df_test.sample(5)
for i, row in sample.iterrows():
    print(row.q)
    print('answer:', row.a)
    print('model: ', answer(row.q))
    print('---')

Минерал, сульфид марганца
answer: алабандин
model:  сульфид
---
Где находится родина табака?
answer: Южная Америка
model:  Бразилия
---
Старинный русский головной убор с приподнятым вверх спереди и сзади околышем
answer: кораблик
model:  шнур
---
Почетное звание у тюрков и монголов, дававшееся за воинские подвиги
answer: батыр
model:  аким
---
Счетный прибор
answer: арифмометр
model:  таблица
---


In [ ]:
for q in ['Животное с хоботом', 'Деревянный дом', 'кто возглавил революцию в России?']:
    print(q)
    print(answer(q, do_sample=True, top_p=0.9))
    print()

Животное с хоботом
хомяк

Деревянный дом
ампир

кто возглавил революцию в России?
Пётр



In [ ]:
new_model_name = 'final_model'  # название папки
model.save_pretrained(new_model_name)
tokenizer.save_pretrained(new_model_name)

('final_model/tokenizer_config.json',
 'final_model/special_tokens_map.json',
 'final_model/spiece.model',
 'final_model/added_tokens.json')

## Выкладка на HG

In [ ]:
# install LFS
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!sudo apt-get install git-lfs
!git lfs install

Detected operating system as Ubuntu/bionic.
Checking for curl...
Detected curl...
Checking for gpg...
Detected gpg...
Running apt-get update... done.
Installing apt-transport-https... done.
Installing /etc/apt/sources.list.d/github_git-lfs.list...done.
Importing packagecloud gpg key... done.
Running apt-get update... done.

The repository is setup! You can now install packages.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 38 not upgraded.
Need to get 6,521 kB of archives.
After this operation, 14.7 MB of additional disk space will be used.
Get:1 https://packagecloud.io/github/git-lfs/ubuntu bionic/main amd64 git-lfs amd64 3.0.1 [6,521 kB]
Fetched 6,521 kB in 0s (19.5 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl

In [ ]:
# install HF
!pip install huggingface_hub

In [ ]:
# login manually
!git config --global credential.helper store
!huggingface-cli login


        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        
Username: cointegrated
Password: 
Login successful
Your token has been saved to /root/.huggingface/token


In [ ]:
# Create the repository
!huggingface-cli repo create rut5-base-quiz

git version 2.17.1
git-lfs/3.0.1 (GitHub; linux amd64; go 1.17)

You are about to create cointegrated/rut5-base-quiz
Proceed? [Y/n] Y

Your repo now lives at:
  https://huggingface.co/cointegrated/rut5-base-quiz

You can clone it locally with the command below, and commit/push as usual.

  git clone https://huggingface.co/cointegrated/rut5-base-quiz



In [ ]:
# initialize it locally
! git clone https://huggingface.co/cointegrated/rut5-base-quiz
! cd rut5-base-quiz &&  git lfs install & git config --global user.email "dale.david@mail.ru"

Cloning into 'rut5-base-quiz'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0)
Unpacking objects: 100% (3/3), done.
Updated git hooks.
Git LFS initialized.


In [ ]:
# copy the model from disk
! cp -r final_model/* rut5-base-quiz

In [ ]:
# add to git and commit the model
! cd rut5-base-quiz && git add . && git commit -m "The first commit" && git push

[main e9a32de] The first commit
 5 files changed, 38 insertions(+)
 create mode 100644 config.json
 create mode 100644 pytorch_model.bin
 create mode 100644 special_tokens_map.json
 create mode 100644 spiece.model
 create mode 100644 tokenizer_config.json
Uploading LFS objects: 100% (2/2), 978 MB | 17 MB/s, done.
Counting objects: 7, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (7/7), done.
Writing objects: 100% (7/7), 1.28 KiB | 1.28 MiB/s, done.
Total 7 (delta 0), reused 0 (delta 0)
To https://huggingface.co/cointegrated/rut5-base-quiz
   05d9f10..e9a32de  main -> main
